In [5]:
from google.colab import drive

ImportError: No module named google.colab

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [184]:
import pandas as pd
import time
import re
import requests
#from urllib.request import urlopen
import urllib3
import bs4
import warnings

In [6]:
def ParserCsvInputBookCrossing(currentLine):
    L = currentLine.split('"')
    
    return [L[i] for i in range(16) if i%2 == 1]

    
#    L = currentLine.split(';')
#    if len(L) == 8:
#        return L
#        
#    else:
#        g = re.search('([0-9X]+);(.+);([0-9]{4,4})(.+)(;http.+.jpg)+(;http.+.jpg)+(;http.+.jpg)+', currentLine)
#
#        g_2 = g[2].split('"')
#        if len(g_2) == 1:
#            g_2_12 = g[2].split(';')
#        else:
#            g_2_12 = [g_2[i] for i in range(len(g_2)) if not (g_2[i] == '' or g_2[i] == ';')]
#     
#        return [x.strip(';') for x in [g[1], g_2_12[0], g_2_12[1], g[3], g[4], g[5], g[6], g[7]]]


In [192]:
def GoogleSearchFunction(isbn = None, title = None, author = None):
    ''' Find information on internet about the book entered in function's arguments 
        Either isbn = ..., Or title = ..., author = ... '''
    
    #3 kinds of books identifiers
    ISBNdict = {"ISBN_10": 0, "ISBN_13": 0, "OtherID": 0}
    
    #all is reset
    ISBNdict["ISBN_10"] = 0.0
    ISBNdict["ISBN_13"] = 0.0   
    ISBNdict["OtherID"] = 0.0
    autGoogle = []
    titleSearched = ''
    autSearched = ''
    pubSearched = ''
    pubData = ''
    catSearched = ''
    descSearched = ''
    langSearched = ''
    imageSearched = ''
    pageSearched = 0

    ISBNfound = False
    AUTHORfound = False
    repGoogle = ''
    
    try:
        #book is searched on internet thanks to its ISBN_10 identifier
        if (isbn is not None):
            
            ISBNdict["ISBN_10"] = isbn

            #Request format for a search on Google Book web site
            #=> We have to format the ISBN so that it is written on 10 digits => {0:0>10s}
            requeteIsbnGoogle = "https://www.googleapis.com/books/v1/volumes?q=isbn:{0:0>10s}".format(ISBNdict["ISBN_10"])
    
            try:
                #In case where a book reference has been found
                repGoogle = pd.read_json(requeteIsbnGoogle)
                ISBNfound = True

            except:
                #In case where a book reference has NOT been found                
                repGoogle = pd.read_json(requeteIsbnGoogle, typ = 'series')
                ISBNfound = False
        
        #book is searched on internet thanks to its title and 1st author
        else:
            if ((title != None) and (author != None)):

                titleSearched = title.encode('ascii', 'ignore').decode('ascii')
                autSearched = author.encode('ascii', 'ignore').decode('ascii')
                #We keep only the author familly name
                autSearchedInternal = autSearched.lower().split()[-1]

                #Request format for a search on Google Book web site
                #=> We must take care of ' (replace('\'', '')) and space (replace(' ','%20'))
                requeteTitleGoogle = "https://www.googleapis.com/books/v1/volumes?q=title:%s" % (titleSearched.replace('\'', '').replace(' ','%20'))
    #           print(requeteTitleGoogle)
                repGoogle = pd.read_json(requeteTitleGoogle) 

                #The Title has been found on Google Books
                #We only analyse the first book's reference retrieved => repTitleGoogle['items'][0]
                if (repGoogle['totalItems'][0] != 1):

                    if ('authors' in repGoogle['items'][0]['volumeInfo']):

                        #All authors proposed by Google
                        autGoogle = repGoogle['items'][0]['volumeInfo']['authors']

                        #Among all authors proposed by Google, we search correspondance with the first provided author
                        j = 0
                        while ((j < len(autGoogle)) and (AUTHORfound == False)):

                            if (autSearchedInternal in autGoogle[j].lower()):
                                AUTHORfound = True  
                                ISBNfound = True

                            j += 1

        #ISBN or title/author has been recognized on Google Books
        if (ISBNfound == True):
            titleSearched = repGoogle['items'][0]['volumeInfo']['title']

            if 'subtitle' in repGoogle['items'][0]['volumeInfo']:
                titleSearched = titleSearched + ' : ' + repGoogle['items'][0]['volumeInfo']['subtitle']

            #'industryIdentifiers' stands for ISBN_10, ISBN_13 or OtherID
            if ('industryIdentifiers' in repGoogle['items'][0]['volumeInfo']):

                #Among all identifiers type proposed by Google, we search "ISBN_10", "ISBN_13" or "OtherID"
                for k in range(len(repGoogle['items'][0]['volumeInfo']['industryIdentifiers'])):

                    if repGoogle['items'][0]['volumeInfo']['industryIdentifiers'][k]['type'] == "ISBN_10":
                        ISBNdict["ISBN_10"] = repGoogle['items'][0]['volumeInfo']['industryIdentifiers'][k]['identifier']
                        ISBNfound = True

                    elif repGoogle['items'][0]['volumeInfo']['industryIdentifiers'][k]['type'] == "ISBN_13":
                        ISBNdict["ISBN_13"] = repGoogle['items'][0]['volumeInfo']['industryIdentifiers'][k]['identifier']
                        ISBNfound = True

                    else:
                        ISBNdict["OtherID"] = repGoogle['items'][0]['volumeInfo']['industryIdentifiers'][k]['identifier']
                        ISBNfound = True


            if ('authors' in repGoogle['items'][0]['volumeInfo']):
                autSearched = repGoogle['items'][0]['volumeInfo']['authors'][0]

            if ('publisher' in repGoogle['items'][0]['volumeInfo']):
                pubSearched = repGoogle['items'][0]['volumeInfo']['publisher']

            if ('categories' in repGoogle['items'][0]['volumeInfo']):
                catSearched = repGoogle['items'][0]['volumeInfo']['categories'][0]

            if ('searchInfo' in repGoogle['items'][0]):
                descSearched = repGoogle['items'][0]['searchInfo']['textSnippet']                    

            if ('language' in repGoogle['items'][0]['volumeInfo']):
                langSearched = repGoogle['items'][0]['volumeInfo']['language']

            if ('imageLinks' in repGoogle['items'][0]['volumeInfo']):
                imageSearched = repGoogle['items'][0]['volumeInfo']['imageLinks']['smallThumbnail']

            if ('pageCount' in repGoogle['items'][0]['volumeInfo']):
                pageSearched = repGoogle['items'][0]['volumeInfo']['pageCount']

            if ('publishedDate' in repGoogle['items'][0]['volumeInfo']):
                pubData = repGoogle['items'][0]['volumeInfo']['publishedDate']
                pubData = re.search("[0-9]{4,4}", pubData).group(0)

        return (ISBNfound, [ISBNdict["ISBN_10"], ISBNdict["ISBN_13"], ISBNdict["OtherID"], titleSearched, autSearched, \
                               pubData, pubSearched, catSearched, descSearched, langSearched, imageSearched, pageSearched, '', '', []])

    except:
        return (False, [ISBNdict["ISBN_10"], ISBNdict["ISBN_13"], ISBNdict["OtherID"], titleSearched, autSearched, \
                           pubData, pubSearched, catSearched, descSearched, langSearched, imageSearched, pageSearched, '', '', []])



In [193]:
def GoodReadsSearchFunction(isbn = None, title = None, author = None):
    ''' Find information on internet about the book entered in function's arguments 
        Either isbn = ..., Or title = ..., author = ... '''
    
    #3 kinds of books identifiers
    ISBNdict = {"ISBN_10": 0, "ISBN_13": 0, "OtherID": 0}
    
    #all is reset
    ISBNdict["ISBN_10"] = 0.0
    ISBNdict["ISBN_13"] = 0.0   
    ISBNdict["OtherID"] = 0.0
    autGoodReads = ''
    titleSearched = ''
    autSearched = ''
    pubSearched = ''
    pubData = ''
    catSearched = ''
    descSearched = ''
    awardsSearched = ''
    languageSearched = ''
    pagesSearched = 0
    authorGenreSearched = ''
    goodreadsLinksSeriesList = []

    AUTHORfound = False
    ISBNfound = False
    
    try:
        if(isbn is not None):
            #https://www.goodreads.com/search?q=195153448&qid=
            ISBNdict["ISBN_10"] = isbn
    
            urlIdBookGoodReads_start = "https://www.goodreads.com/search?q={0:0>10s}".format(ISBNdict["ISBN_10"])
            urlIdBookGoodReads_end = '&qid='
            bookUrlForSearch = urlIdBookGoodReads_start + urlIdBookGoodReads_end
            
            #source = urlopen(bookUrlForSearch)
            #source = requests.get(bookUrlForSearch).content
            http = urllib3.PoolManager()
            source = http.request('GET', bookUrlForSearch).data
            soup = bs4.BeautifulSoup(source, 'html.parser')
            ISBNfound = (soup.find("h1", id="bookTitle") != None)

        else:    
            if ((title != None) and (author != None)):
                urlIdBookGoodReads_start = 'https://www.goodreads.com/search?utf8=%E2%9C%93&q='
                urlIdBookGoodReads_end = '&search_type=books'
                
                titleSearched = title.encode('ascii', 'ignore').decode('ascii')
                autSearched = author.encode('ascii', 'ignore').decode('ascii')
                #We keep only the author familly name
                autSearchedInternal = autSearched.lower().split()[-1]
                
                #First we have to find the book page: we need GoodReads identifier of the book 
                bookUrlForSearch = urlIdBookGoodReads_start + titleSearched.replace(' ', '+') + '+' + autSearchedInternal.replace(' ', '+') + urlIdBookGoodReads_end
#                print(bookUrlForSearch)
                
                #sourceIdBookGoodReads = urlopen(bookUrlForSearch)
                sourceIdBookGoodReads = requests.get(bookUrlForSearch).content
                soupIdBookGoodReads = bs4.BeautifulSoup(sourceIdBookGoodReads, 'html.parser')
                
                #Search of the reference compliant with title and author provided, among all proposed books references
                booksReference = soupIdBookGoodReads.find_all("a", class_="bookTitle")
                j = 0
                while ((j < len(booksReference)) and (AUTHORfound == False)):
                    autGoodReads = booksReference[j].find_next("a", class_="authorName").text
                    
                    if (autSearchedInternal in autGoodReads.lower()):
                        AUTHORfound = True
                    j += 1    
                
                #Now we go to the selected book's page
                if (AUTHORfound == True):
                    #source = urlopen('https://www.goodreads.com' + booksReference[j-1]["href"]) 
                    source = requests.get('https://www.goodreads.com' + booksReference[j-1]["href"]).content
                    soup = bs4.BeautifulSoup(source, 'html.parser')
                    ISBNfound = True

        #Now we are on the specific book's web page
        #=> Common analysis begins
        if ISBNfound:
            titleSearched = soup.find("h1", id="bookTitle").text
            titleSearched = titleSearched.replace('\n', '').strip()
            
            #Search of ISBN (10 and / or 13), language and awards
            searchForISBN_Lang_Awards_Series = soup.find("div", id="bookDataBox").find_all("div", class_="clearFloats")

            for i in range(len(searchForISBN_Lang_Awards_Series)):
                
                if searchForISBN_Lang_Awards_Series[i].find("div", "infoBoxRowTitle").text == 'ISBN':
                    isbnFull = searchForISBN_Lang_Awards_Series[i].find("div", "infoBoxRowItem").text.strip()

                    ISBNdict["ISBN_10"] = isbnFull.split("(ISBN13:")[0].strip()
                    
                if searchForISBN_Lang_Awards_Series[i].find("div", "infoBoxRowTitle").text == 'Edition Language':
                    languageSearched = searchForISBN_Lang_Awards_Series[i].find("div", "infoBoxRowItem").text
                    #In order to be compliant with GoogleBooks names
                    if (languageSearched == 'English'):
                        languageSearched = 'en'
            
                if searchForISBN_Lang_Awards_Series[i].find("div", "infoBoxRowTitle").text == 'Literary Awards':
                    awardsSearched = searchForISBN_Lang_Awards_Series[i].find("div", "infoBoxRowItem").text
                    awardsSearched = awardsSearched.strip()
            
            #Search of published date and publisher       
            searchForPublished = soup.find("div", id="details").find_all("div", class_="row")

            for i in range(len(searchForPublished)):
                if 'Published' in searchForPublished[i].text:
                    #Current published date and additional information on first publication date is also provided
                    if (searchForPublished[i].find("nobr", class_="greyText")):
                        infoFirstPublished = searchForPublished[i].find("nobr", class_="greyText")
                        tempPublished = searchForPublished[i].text.split(infoFirstPublished.text)
                        tempPublished = tempPublished[0].strip().replace('\n', '').replace('Published', '').split('by')
                        pubData = tempPublished[0].strip()
                        pubData = re.search("[0-9]{4,4}", pubData).group(0)
                        pubSearched = tempPublished[1].strip()
                    #Only current published date is provided
                    else:
                        tempPublished = searchForPublished[i].text.strip().replace('\n', '').replace('Published', '').split('by')
                        pubData = tempPublished[0].strip()
                        pubData = re.search("[0-9]{4,4}", pubData).group(0)
                        pubSearched = tempPublished[1].strip()

            #Search of the pages number
            searchForPages = soup.find("div", id="details").find_all("div", class_="row")
            for i in range(len(searchForPages)):
                if 'pages' in searchForPages[i].text:
                    pagesSearched = searchForPublished[i].text
                    pagesSearched = pagesSearched.split(',')[-1].split('pages')[0].strip()
        
            #Search for the book's genre
            if (soup.find("a", class_="actionLinkLite bookPageGenreLink")):
                catSearched = soup.find_all("a", class_="actionLinkLite bookPageGenreLink")[0].text
            
            #Search for knowing if book is part of a serie
            #=> If yes, we keep the URL of the other mentionned books of the serie
            if soup.find("div", class_="seriesList"):
                if (soup.find("div", class_="seriesList").find_next("h2").text == 'Other books in the series'):
                    #We go to this specific book serie web page
                    linkTowardSeries = soup.find("div", class_="seriesList").find_next("h2").find_next("a")["href"]
                    r = requests.get('https://www.goodreads.com' + linkTowardSeries)
                    
                    #We keep all the other books reference of this serie
                    seriesList = re.findall('href="/book/show/[0-9]+', str(r.content))
                    goodreadsLinksSeriesList = [seriesList[i].replace('href="', 'https://www.goodreads.com') for i in range(len(seriesList)) if i%2 == 0]
            
            #search of the author genre: we need to finf the URL of author page first
            autSearched = soup.find("a", class_="authorName").text
            #sourceAuthor = urlopen(soup.find("a", class_="authorName")["href"])
            sourceAuthor = requests.get(soup.find("a", class_="authorName")["href"]).content
            soupAuthor = bs4.BeautifulSoup(sourceAuthor, 'html.parser')
            
            #On the author web site page, we search for the author genre
            searchForAuthorGenre = soupAuthor.find_all("div", class_="dataTitle")
            for i in range(len(searchForAuthorGenre)):
                if (searchForAuthorGenre[i].text == "Genre"):
                    authorGenreSearched = searchForAuthorGenre[i].find_next("div", class_="dataItem").text.split(',')[0]
                    authorGenreSearched = authorGenreSearched.replace('\n', '').strip()
    
        return (ISBNfound, [ISBNdict["ISBN_10"], ISBNdict["ISBN_13"], ISBNdict["OtherID"], titleSearched, autSearched, \
                pubData, pubSearched, catSearched, descSearched, languageSearched, '', pagesSearched, awardsSearched, \
                authorGenreSearched, goodreadsLinksSeriesList])
    
    except:
        return (False, [ISBNdict["ISBN_10"], ISBNdict["ISBN_13"], ISBNdict["OtherID"], titleSearched, autSearched, \
                pubData, pubSearched, catSearched, descSearched, languageSearched, '', pagesSearched, awardsSearched, \
                authorGenreSearched, goodreadsLinksSeriesList])    


In [194]:
def GoodReadsPartialSearchFunction(isbn =  None):
    
    awardsSearched = ''
    authorGenreSearched = ''
    goodreadsLinksSeriesList = []
    
    ISBNfound = False
    
    try:
        urlIdBookGoodReads_start = "https://www.goodreads.com/search?q={0:0>10s}".format(isbn)
        urlIdBookGoodReads_end = '&qid='
        bookUrlForSearch = urlIdBookGoodReads_start + urlIdBookGoodReads_end
        
        #source = urlopen(bookUrlForSearch)
        source = requests.get(bookUrlForSearch).content
        soup = bs4.BeautifulSoup(source, 'html.parser')
        ISBNfound = (soup.find("h1", id="bookTitle") != None)


        #Now we are on the specific book's web page
        if ISBNfound:
            searchForISBN_Lang_Awards_Series = soup.find("div", id="bookDataBox").find_all("div", class_="clearFloats")
            
            for i in range(len(searchForISBN_Lang_Awards_Series)):
                if searchForISBN_Lang_Awards_Series[i].find("div", "infoBoxRowTitle").text == 'Literary Awards':
                    awardsSearched = searchForISBN_Lang_Awards_Series[i].find("div", "infoBoxRowItem").text

                if searchForISBN_Lang_Awards_Series[i].find("div", "infoBoxRowTitle").text == 'Edition Language':
                    languageSearched = searchForISBN_Lang_Awards_Series[i].find("div", "infoBoxRowItem").text
                    #In order to be compliant with GoogleBooks names
                    if (languageSearched == 'English'):
                        languageSearched = 'en'

            #Search for knowing if book is part of a serie
            #=> If yes, we keep the URL of the other mentionned books of the serie
            if soup.find("div", class_="seriesList"):
                if (soup.find("div", class_="seriesList").find_next("h2").text == 'Other books in the series'):
                    #We go to this specific book serie web page
                    linkTowardSeries = soup.find("div", class_="seriesList").find_next("h2").find_next("a")["href"]
                    r = requests.get('https://www.goodreads.com' + linkTowardSeries)
                    
                    #We keep all the other books reference of this serie
                    seriesList = re.findall('href="/book/show/[0-9]+', str(r.content))
                    goodreadsLinksSeriesList = [seriesList[i].replace('href="', 'https://www.goodreads.com') for i in range(len(seriesList)) if i%2 == 0]
                    
            #search of the author genre: we need to finf the URL of author page first
            #sourceAuthor = urlopen(soup.find("a", class_="authorName")["href"])
            sourceAuthor = requests.get(soup.find("a", class_="authorName")["href"]).content
            soupAuthor = bs4.BeautifulSoup(sourceAuthor, 'html.parser')
            
            #On the author web site page, we search for the author genre
            searchForAuthorGenre = soupAuthor.find_all("div", class_="dataTitle")
            for i in range(len(searchForAuthorGenre)):
                if (searchForAuthorGenre[i].text == "Genre"):
                    authorGenreSearched = searchForAuthorGenre[i].find_next("div", class_="dataItem").text.split(',')[0]
                    authorGenreSearched = authorGenreSearched.replace('\n', '').strip()
    
        return (ISBNfound, [awardsSearched, authorGenreSearched, goodreadsLinksSeriesList])
    
    except:
        return (False, [awardsSearched, authorGenreSearched, goodreadsLinksSeriesList])    


In [195]:
def internetSearch(f, SaveFileName, theColumns, ind_start, ind_end):

    NotFoundBooks = 0    
    pdT = pd.DataFrame(columns = theColumns, dtype = object)
    pdT['Same serie'] = pdT['Same serie'].astype(object)

    for i in range(0, ind_start-1): 
        f.readline()
        
    csvLineParsed = ParserCsvInputBookCrossing(f.readline())
    ret, refBook = GoogleSearchFunction(isbn = csvLineParsed[0])
    pdT.loc[0] = refBook 
    pdT.loc[0:1].to_csv(SaveFileName, sep = ';', index = False, mode = 'w')
    
    #it doesn't start at 0 so that the trigger index 'len(pdT) - savingRate:len(pdT)+1' of save is correct
    for i in range(ind_start+1, ind_end):    
        retGoogle = False
        retGoodReads = False
        retPartial = False

        lili = f.readline()
        csvLineParsed = ParserCsvInputBookCrossing(lili)
        #print(csvLineParsed)
        
        #First we use Google Books web site: search fastest
        retGoogle, refBook = GoogleSearchFunction(isbn = csvLineParsed[0])
        
        if (retGoogle == False):
            #if ISBN not found on Google Books, we try on Google Reads
            retGoodReads, refBook = GoodReadsSearchFunction(isbn = csvLineParsed[0])
            
            if (retGoodReads == False):
                #if ISBN not found (on Google Books and on Good Reads), 
                #we try search on GoodReads only a search with title and author 
                retGoodReads, refBook = GoodReadsSearchFunction(title = csvLineParsed[1], author = csvLineParsed[2].split(';')[-1])
                if (retGoodReads == False):
                    NotFoundBooks += 1
                
        else:
           
            #If ISBN found on Google Books, we complete some columns with GoodReads search
            retPartial, refBookPartial = GoodReadsPartialSearchFunction(isbn = csvLineParsed[0])
            if (retPartial == True):
                refBook[-3:] = refBookPartial

        pdT = pd.DataFrame(columns = theColumns, dtype = object)
        pdT.loc[0] = refBook
        pdT.loc[0:1].to_csv(SaveFileName, sep = ';', index = False, mode = 'a', header = False, encoding='utf-8')
        #Google doesn't like when too much requests are performed...
        #time.sleep(10.0)
        
    return NotFoundBooks


In [196]:
import io

#cheminBookCrossing = '/content/drive/MyDrive/ProjetML/Base_BookCrossing/Base_originale/'
cheminBookCrossing = '/home/ubuntu/BooksProject_GitHub/Books_project/datasets/'

#cheminCommon = '/content/drive/MyDrive/ProjetML/Base_BookCrossing/Base_completee/'
cheminCommon = '/home/ubuntu/BooksProject_GitHub/Books_project/datasets/'

SaveFileName = cheminCommon + 'bothWebSites_InternetSearch_21_03_2021_30418___.csv'
        
theColumns = ['ISBN_10', 'ISBN_13', 'OtherID', 'Book-Title', 'Book-Author', \
                            'Year-Of-Publication', 'Publisher', 'Category', 'Description', 'Language', \
                            'Image', 'Pages', 'Awards', "Author's genre", 'Same serie']
    
#List of books to search on Google
#f = open(cheminBookCrossing + 'BX-Books.csv', encoding="utf8", errors="replace")
f = io.open(cheminBookCrossing + 'BX-Books.csv', encoding="utf8", errors="replace")
f.readline()
        
#Internet search function
with warnings.catch_warnings():
    warnings.warn("Let this be your last warning")
    warnings.simplefilter("ignore")
    #NotFoundBooks = internetSearch(f, SaveFileName, theColumns, 30418, 270000) 
    NotFoundBooks = internetSearch(f, SaveFileName, theColumns, 0, 15) 
    
print("Not found books: %d" % NotFoundBooks)
f.close()   


/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:20: UserWarning: Let this be your last warning


"0002005018";"Clara Callan";"Richard Bruce Wright";"2001";"HarperFlamingo Canada";"http://images.amazon.com/images/P/0002005018.01.THUMBZZZ.jpg";"http://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg";"http://images.amazon.com/images/P/0002005018.01.LZZZZZZZ.jpg"

"0060973129";"Decision in Normandy";"Carlo D'Este";"1991";"HarperPerennial";"http://images.amazon.com/images/P/0060973129.01.THUMBZZZ.jpg";"http://images.amazon.com/images/P/0060973129.01.MZZZZZZZ.jpg";"http://images.amazon.com/images/P/0060973129.01.LZZZZZZZ.jpg"

"0374157065";"Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It";"Gina Bari Kolata";"1999";"Farrar Straus Giroux";"http://images.amazon.com/images/P/0374157065.01.THUMBZZZ.jpg";"http://images.amazon.com/images/P/0374157065.01.MZZZZZZZ.jpg";"http://images.amazon.com/images/P/0374157065.01.LZZZZZZZ.jpg"

"0393045218";"The Mummies of Urumchi";"E. J. W. Barber";"1999";"W. W. Norton &amp; Company";"http://images.am

In [22]:
import os
os.getcwd()


'/home/ubuntu/BooksProject_GitHub/Books_project/WebScrapper'

In [137]:
"
                  1552041778
                      (ISBN13: 9781552041772)".split("ISBN13")

SyntaxError: EOL while scanning string literal (<ipython-input-137-f3ad4978c153>, line 1)

In [140]:
test ="tot"
test.split("ISBN")

['tot']

In [ ]:
http = urllib3.PoolManager()
source = http.request('GET', https://www.goodreads.com/search?q=1552041778&qid=).data
soup = bs4.BeautifulSoup(source, 'html.parser')
ISBNfound = (soup.find("h1", id="bookTitle") != None)

In [152]:
http = urllib3.PoolManager()


In [154]:
source = http.request('GET', "https://www.goodreads.com/search?q=1552041778&qid=").data

In [155]:
soup = bs4.BeautifulSoup(source, 'html.parser')

In [156]:
(soup.find("h1", id="bookTitle") != None)

True

In [159]:
searchForPublished = soup.find("div", id="details").find_all("div", class_="row")

In [160]:
searchForPublished

[<div class="row"><span itemprop="bookFormat">Audio Cassette</span>, <span itemprop="bookEdition">Abridged</span>, <span itemprop="numberOfPages">0 pages</span></div>,
 <div class="row">\n            Published\n        June 1st 1999\n         by Mira Books\n\n      </div>]

In [178]:
searchForPublished[1].find("nobr", class_="greyText")

In [172]:
searchForPublished[1].text.strip().replace('\n', '').replace('Published', '').split('by')

[u'        June 1st 1999         ', u' Mira Books']

In [175]:
temp =searchForPublished[1].text.strip().replace('\n', '').replace('Published', '').split('by')[0].strip()

In [176]:
temp

u'June 1st 1999'

In [177]:
re.search("[0-9]{4,4}", temp).group(0)

u'1999'

In [ ]:
                    if (searchForPublished[i].find("nobr", class_="greyText")):
                        infoFirstPublished = searchForPublished[i].find("nobr", class_="greyText")
                        tempPublished = searchForPublished[i].text.split(infoFirstPublished.text)
                        tempPublished = tempPublished[0].strip().replace('\n', '').replace('Published', '').split('by')
                        pubData = tempPublished[0].strip()
                        pubData = re.search("[0-9]{4,4}", pubData)[0]
                        pubSearched = tempPublished[1].strip()
                    #Only current published date is provided
                    else:
                        tempPublished = searchForPublished[i].text.strip().replace('\n', '').replace('Published', '').split('by')
                        pubData = tempPublished[0].strip()
                        pubData = re.search("[0-9]{4,4}", pubData)[0]
                        pubSearched = tempPublished[1].strip()